### (Optional) Uncomment below to use Google Drive as a workspace directory

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ln -sfn /content/drive/MyDrive/elephant_workspace /workspace

# Download ELEPHANT server

In [ ]:
%env ELEPHANT_SERVER_VERSION = v0.2.1-dev

In [ ]:
!wget -q -c -nc https://github.com/elephant-track/elephant-server/archive/refs/tags/$ELEPHANT_SERVER_VERSION.tar.gz
!mkdir -p /opt/elephant
!tar -zxf $ELEPHANT_SERVER_VERSION.tar.gz --strip-components=1 -C /opt/elephant
!rm $ELEPHANT_SERVER_VERSION.tar.gz

# Set up conda environment

In [ ]:
# Acknowledgements:
#     https://donaldsrepo.github.io/Notebooks/GoogleColabCondaCreateEnv.html
conda_path = ''
try:
    conda_path = !which conda
finally:
    print('')

if (len(conda_path) == 0):
    print('installing miniconda')
    !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    !bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local
    !rm Miniconda3-4.5.4-Linux-x86_64.sh
    !conda update conda -y -q
    !source /usr/local/etc/profile.d/conda.sh
    !conda init
    !conda install -c conda-forge -y mamba
    !mamba clean -qafy
else:
    print('found miniconda')

!mamba env update -f /opt/elephant/environment.yml


# Install apt packages

In [ ]:
!apt-get update && apt-get install --no-install-recommends --no-install-suggests -y \
    nginx \
    redis-server \
    supervisor \
    ca-certificates \
    curl \
    gnupg \
    gosu \
    openssh-server \
    pwgen

# Install RabbitMQ

In [ ]:
%%bash
apt-get install curl gnupg debian-keyring debian-archive-keyring apt-transport-https --no-install-recommends --no-install-suggests -y

## Ignore warnings
export APT_KEY_DONT_WARN_ON_DANGEROUS_USAGE=1
## Team RabbitMQ's main signing key
apt-key adv --keyserver "hkps://keys.openpgp.org" --recv-keys "0x0A9AF2115F4687BD29803A206B73A36E6026DFCA"
## Cloudsmith: modern Erlang repository
curl -1sLf https://dl.cloudsmith.io/public/rabbitmq/rabbitmq-erlang/gpg.E495BB49CC4BBE5B.key | apt-key add -
## Cloudsmith: RabbitMQ repository
curl -1sLf https://dl.cloudsmith.io/public/rabbitmq/rabbitmq-server/gpg.9F4587F226208342.key | apt-key add -

## Add apt repositories maintained by Team RabbitMQ
tee /etc/apt/sources.list.d/rabbitmq.list <<EOF
## Provides modern Erlang/OTP releases
##
deb https://dl.cloudsmith.io/public/rabbitmq/rabbitmq-erlang/deb/ubuntu bionic main
deb-src https://dl.cloudsmith.io/public/rabbitmq/rabbitmq-erlang/deb/ubuntu bionic main

## Provides RabbitMQ
##
deb https://dl.cloudsmith.io/public/rabbitmq/rabbitmq-server/deb/ubuntu bionic main
deb-src https://dl.cloudsmith.io/public/rabbitmq/rabbitmq-server/deb/ubuntu bionic main
EOF

## Update package indices
apt-get update

## Install Erlang packages
apt-get install --no-install-recommends --no-install-suggests -y erlang-base \
    erlang-asn1 erlang-crypto erlang-eldap erlang-ftp erlang-inets \
    erlang-mnesia erlang-os-mon erlang-parsetools erlang-public-key \
    erlang-runtime-tools erlang-snmp erlang-ssl \
    erlang-syntax-tools erlang-tftp erlang-tools erlang-xmerl

## Install rabbitmq-server and its dependencies
apt-get install rabbitmq-server --no-install-recommends --no-install-suggests -y --fix-missing

# Set up services

In [ ]:
# Set up RabbitMQ
%env RABBITMQ_USER=user
%env RABBITMQ_PASSWORD=user
%env RABBITMQ_PID_FILE=/var/lib/rabbitmq/mnesia/rabbitmq.pid
!cp /opt/elephant/docker/rabbitmq.sh /rabbitmq.sh
!sed -i 's;--timeout 60 ;;g' /rabbitmq.sh
!chmod +x /rabbitmq.sh
# Set up nginx
!cp /opt/elephant/docker/nginx.conf /etc/nginx/nginx.conf
!groupadd nginx && useradd -g nginx nginx
!ln -sf /dev/stdout /var/log/nginx/access.log
!ln -sf /dev/stderr /var/log/nginx/error.log
# Set up uWSGI
!mkdir -p /etc/uwsgi
!cp /opt/elephant/docker/uwsgi.ini /etc/uwsgi/uwsgi.ini
!cp -a /opt/elephant/app /app
%env UWSGI_UID=root
%env UWSGI_GID=root
# Set up supervisor
!cp /opt/elephant/docker/supervisord.conf /etc/supervisor/conf.d/supervisord.conf
!sed -i 's;/opt/conda/bin/uwsgi;uwsgi;g' /etc/supervisor/conf.d/supervisord.conf
!printf "\
\n\
[program:sshd]\n\
command=/usr/sbin/sshd -D\n\
autorestart=true\
" >> /etc/supervisor/conf.d/supervisord.conf

# Set up sshd
!mkdir -p /var/run/sshd
!grep -qxF "PermitRootLogin yes" /etc/ssh/sshd_config || echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!grep -qxF "PasswordAuthentication yes" /etc/ssh/sshd_config || echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

# Download ngrok

In [ ]:
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip -d /opt/ngrok
!rm ngrok-stable-linux-amd64.zip

# Launch tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /workspace/logs

# Start services

In [ ]:
# Acknowledgements:
# @Tamlyn https://stackoverflow.com/a/53252985
import getpass
import json
import random
import string
import urllib.request
import time

#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Set root password
!echo root:$password | chpasswd

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('/opt/ngrok/ngrok authtoken $authtoken && /opt/ngrok/ngrok tcp 22 &')

#Get public address and print connect command
time.sleep(1)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  for tunnel in data['tunnels']:
    if tunnel['name'] == 'command_line':
      (host, port) = tunnel['public_url'][6:].split(':')
      print('*** SSH information *** ')
      print(f'SSH user: root\tSSH host: {host}\tSSH port: {port}')
      print(f'Root password: {password}')
      print('*** For command line interface *** ')
      print('SSH port forward commands (Windows):')
      print(f'\tssh.exe -N -L 8080:localhost:80 -o PubkeyAuthentication=no -o TCPKeepAlive=yes -o ServerAliveInterval=30 -p{port} root@{host}')
      print(f'\tssh.exe -N -L 5672:localhost:5672 -o PubkeyAuthentication=no -o TCPKeepAlive=yes -o ServerAliveInterval=30 -p{port} root@{host}')
      print('SSH port forward commands (Unix):')
      print(f'\tssh -N -L 8080:localhost:80 -o PubkeyAuthentication=no -o TCPKeepAlive=yes -o ServerAliveInterval=30 -p{port} root@{host}')
      print(f'\tssh -N -L 5672:localhost:5672 -o PubkeyAuthentication=no -o TCPKeepAlive=yes -o ServerAliveInterval=30 -p{port} root@{host}')

# Run services
%cd /app
!unlink /var/run/supervisor.sock
for pname in ['rabbitmq', 'epmd', 'uwsgi', 'nginx', 'redis-server', 'sshd']:
  !pkill -9 $pname
!/usr/bin/supervisord